In [18]:
#Credit: https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from sklearn.cross_validation import train_test_split
from collections import defaultdict
from pattern.en import parse
from pattern.en import pprint
from pattern.vector import stem, PORTER, LEMMA
from sklearn.feature_extraction import text 
import re
#import findspark
#findspark.init()
#import pyspark
#from pyspark.sql import SQLContext
import nltk
#nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import ward, dendrogram
stopwords=text.ENGLISH_STOP_WORDS

#DATAFILEPATH='/Users/Dilip_MBP/Documents/CS109/PROJECT/data/'
DATAFILEPATH='C:/Users/balachandrd/Documents/Personal/CS-109/Project/project_cs109/data/'

In [20]:
punctuation = list('.,;:!?()[]{}`''\"@#$^&*+-|=~_')
regex1=re.compile(r"\.{2,}")
regex2=re.compile(r"\-{2,}")
regex3=re.compile(r"\\\\")
stopwords=text.ENGLISH_STOP_WORDS
CUSTOM_STOPWORDS=['']

In [6]:
# Read and store csv data in to Master index

bike_masterdf = pd.read_csv(DATAFILEPATH+'bike_details.csv',encoding='utf-8')
print "Number of samples in the database:", bike_masterdf.shape[0]
bike_masterdf.head(2)

Number of samples in the database: 59619


,Unnamed: 0,id,title,serial,manufacturer_name,frame_model,year,thumb,large_img,is_stock_img,stolen,stolen_location,date_stolen,registration_created_at,registration_updated_at,url,api_url,manufacturer_id,paint_description,name,frame_size,description,rear_tire_narrow,front_tire_narrow,type_of_cycle,test_bike,rear_wheel_size_iso_bsd,front_wheel_size_iso_bsd,handlebar_type_slug,frame_material_slug,front_gear_type_slug,rear_gear_type_slug,stolen_id,s_date_stolen,s_location,latitude,longitude,theft_description,locking_description,lock_defeat_description,police_report_number,police_report_department,s_rec_created_at,create_open311,sder_formatted_address,sder_street_number,sder_route,sder_postal_code,sder_neighborhood,sder_city,sder_county,sder_state,sder_country,date_stolen_epoch,registration_created_at_epoch,registration_updated_at_epoch,s_rec_created_at_epoch
0,0,50088,Schwinn Gateway,absent,Schwinn,Gateway,NaN,NaN,NaN,False,False,NaN,1970-01-01 00:00:00,2015-07-06 22:27:01,2015-11-03 05:36:54,https://bikeindex.org/bikes/50088,https://bikeindex.org/api/v1/bikes/50088,117,NaN,NaN,NaN,NaN,True,NaN,Bike,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1436221621,1446529014,0
1,1,60140,2016 Specialized Diverge Elite DSW,WSBC601010269K,Specialized,Diverge Elite DSW,2016,NaN,NaN,False,False,NaN,1970-01-01 00:00:00,2015-10-08 19:18:50,2015-11-03 05:34:30,https://bikeindex.org/bikes/60140,https://bikeindex.org/api/v1/bikes/60140,307,NaN,NaN,52cm,NaN,True,NaN,Bike,False,NaN,NaN,NaN,aluminum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1444331930,1446528870,0


In [7]:
# Remove nulls in theft description - only stolen bikes have theft description -> store in filtered dataframe
num_stolen = len(bike_masterdf.stolen[bike_masterdf.stolen == True])
print "Number of bikes reported as stolen: ", num_stolen
print "Proportion of bikes reported as stolen:  %.2f" % (float(num_stolen)/float(bike_masterdf.shape[0]))
desc_mask = bike_masterdf.theft_description.isnull()
bike_subdf = bike_masterdf[~desc_mask]
print "Number of Stolen bikes with theft description:", bike_subdf.shape[0]

bike_subdf.to_csv(DATAFILEPATH+'stolen_bikes.csv',encoding='utf-8')
bike_subdf.head(2)

Number of bikes reported as stolen:  33182
Proportion of bikes reported as stolen:  0.56
Number of Stolen bikes with theft description: 29838


,Unnamed: 0,id,title,serial,manufacturer_name,frame_model,year,thumb,large_img,is_stock_img,stolen,stolen_location,date_stolen,registration_created_at,registration_updated_at,url,api_url,manufacturer_id,paint_description,name,frame_size,description,rear_tire_narrow,front_tire_narrow,type_of_cycle,test_bike,rear_wheel_size_iso_bsd,front_wheel_size_iso_bsd,handlebar_type_slug,frame_material_slug,front_gear_type_slug,rear_gear_type_slug,stolen_id,s_date_stolen,s_location,latitude,longitude,theft_description,locking_description,lock_defeat_description,police_report_number,police_report_department,s_rec_created_at,create_open311,sder_formatted_address,sder_street_number,sder_route,sder_postal_code,sder_neighborhood,sder_city,sder_county,sder_state,sder_country,date_stolen_epoch,registration_created_at_epoch,registration_updated_at_epoch,s_rec_created_at_epoch
3,3,11542,2013 Trek 4300,WTU124C5170G,Trek,4300,2013,NaN,NaN,False,True,"San Jose, CA, 95126",2013-06-10 06:00:00,2013-08-20 06:00:00,2015-11-03 05:11:14,https://bikeindex.org/bikes/11542,https://bikeindex.org/api/v1/bikes/11542,47,NaN,NaN,NaN,"Trek 4300, Mountain bicycle, disc brakes, blac...",True,NaN,Bike,False,NaN,NaN,NaN,NaN,NaN,NaN,7889,1370844000,"San Jose, CA, 95126",37.329012,-121.916021,bike rack,NaN,NaN,T13006853,San Jose,2014-06-14 17:52:49,False,"1323 Martin Ave, San Jose, CA 95126, USA",1323,Martin Avenue,95126,Shasta-Hanchett Park,San Jose,Santa Clara County,California,United States,1370844000,1376978400,1446527474,1402768369
4,4,13359,2010 Schwinn Katana,absent,Schwinn,Katana,2010,https://files.bikeindex.org/uploads/Pu/4903/sm...,https://files.bikeindex.org/uploads/Pu/4903/la...,False,True,"Sandusky, OH, 44870",2010-07-10 06:00:00,2010-07-11 06:00:00,2015-11-03 05:17:53,https://bikeindex.org/bikes/13359,https://bikeindex.org/api/v1/bikes/13359,117,NaN,NaN,58cm,"The bike is a grey, blue, and black Schwinn Ka...",True,NaN,Bike,False,NaN,NaN,NaN,NaN,NaN,NaN,9705,1278741600,"Sandusky, OH, 44870",41.440356,-82.768114,car bike rack Reward: $75,NaN,NaN,2010004433,Sandusky,2014-06-14 18:17:08,False,"4614 Venice Heights Blvd, Sandusky, OH 44870, USA",4614,Venice Heights Boulevard,44870,NaN,Sandusky,Erie County,Ohio,United States,1278741600,1278828000,1446527873,1402769828


In [17]:
# Create function to parse incoming text, tokenize ALL words - no filtering 
regex1=re.compile(r"\.{2,}")
regex2=re.compile(r"\-{2,}")
regex3=re.compile(r"\\\\")

stemmer = SnowballStemmer("english")

#use nltk tokenizer to tokenize and stem words in incoming sentence 
def clean_and_tokenize(intext):
    intext=re.sub(regex1, ' ', intext)
    intext=re.sub(regex2, ' ', intext)
    intext=re.sub(regex3, ' ', intext)
    
    #tokenize sentence
    dirty_tokens = [word.lower() for sent in nltk.sent_tokenize(intext) for word in nltk.word_tokenize(sent)]
    tokens=[]
    for eacht in dirty_tokens:
        if eacht not in punctuation:
            tokens.append(eacht)
    stemmed_tokens = [stemmer.stem(t) for t in tokens]
    
    return " ".join(stemmed_tokens)
    
    


In [14]:
# Create function to parse incoming text, tokenize and check for  ******* Nouns
regex1=re.compile(r"\.{2,}")
regex2=re.compile(r"\-{2,}")
regex3=re.compile(r"\\\\")
punctuation = list('.,;:!?()[]{}`''\"@#$^&*+-|=~_')

def gather_nouns(thetext):
    thetext=re.sub(regex1, ' ', thetext)
    thetext=re.sub(regex2, ' ', thetext)
    thetext=re.sub(regex3, ' ', thetext)

    nouns=[]
    for sentence in parse(thetext, tokenize=True, lemmata = True, encoding = 'utf-8').split():
        for token in sentence:
            #print token
            if len(token[4]) >0:
                if token[1] in ['NN', 'NNS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    nouns.append(token[4])
    nouns2=[]

    for n in nouns:
        if len(n)!=0:
            nouns2.append(n)
        else:
            print "************** WE HAVE A PROBLEM **************"
    return ' '.join(nouns2)

In [15]:
def gather_nouns_and_adj(thetext):
    thetext=re.sub(regex1, ' ', thetext)
    thetext=re.sub(regex2, ' ', thetext)
    thetext=re.sub(regex3, ' ', thetext)

    tokens=[]
    for sentence in parse(thetext, tokenize=True, lemmata = True, encoding = 'utf-8').split():
        for token in sentence:
            #print token
            if len(token[4]) >0:
                if token[1] in ['NN','NNS','JJ','JJR','JJS' ]:
                    if token[4] in CUSTOM_STOPWORDS or token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    
                    #print token[4]    
                    tokens.append(token[4])
    tokens2=[]

    for n in tokens:
        if len(n)!=0:
            tokens2.append(n)
            # Remove repeating bigrams (e.g bike bike)
            if (len(tokens2) !=1):
                if (tokens2[-1] == tokens2[-2]):
                    tokens2.pop()
            
            
                            
        else:
            print "************** WE HAVE A PROBLEM **************"
    return ' '.join(tokens2)

In [21]:
atext = "The bikes were locked to a sign pole in my apartment by the corner of the superstore. The lock was a combination lock and i had \
tied chained my helmet to the bike. We have fairly good security in the block of apartments here. The bike was of make 'GT' with shimano\
gears and had a custom seat. The apartment's security office told me that i have no chance of retrieving the bike. My counsin' bike was\
stolen from their home's open garage overnight"

print clean_and_tokenize(atext)
print "\n"
print gather_nouns(atext)
print "\n"
print gather_nouns_and_adj(atext)

the bike were lock to a sign pole in my apart by the corner of the superstor the lock was a combin lock and i had tie chain my helmet to the bike we have fair good secur in the block of apart here the bike was of make gt ' with shimanogear and had a custom seat the apart 's secur offic told me that i have no chanc of retriev the bike my counsin ' bike wasstolen from their home 's open garag overnight


bike sign pole apartment corner superstore lock combination lock helmet bike security block apartment bike make shimanogear custom seat apartment security office chance bike counsin bike home garage


bike sign pole apartment corner superstore lock combination lock helmet bike good security block apartment bike make shimanogear custom seat apartment security office chance bike counsin bike home open garage


In [22]:
#bike_subdf['Clean_theft_description'] = bike_subdf.theft_description.map(clean_and_tokenize)
bike_subdf['Noun_parsed_theft_description'] = bike_subdf.theft_description.map(gather_nouns)
bike_subdf['Noun_adj_parsed_theft_description'] = bike_subdf.theft_description.map(gather_nouns)
#bike_subdf.to_csv(DATAFILEPATH+'stolen_bikes_tokenized_nouns.csv',encoding='utf-8')

C:\Users\balachandrd\AppData\Local\Continuum\Anaconda\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\balachandrd\AppData\Local\Continuum\Anaconda\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [23]:
#Tokenize each theft description to return a list of nouns, proper nouns etc for each sample (row)
# We will remove words whose word count is less than 2
vectorizer = CountVectorizer(min_df=2, stop_words='english')
vectorizer_bigram = CountVectorizer(min_df=2, ngram_range=(2,2), stop_words='english')
text_sparse_matrix = vectorizer.fit_transform(bike_subdf.Noun_adj_parsed_theft_description)    #************* May need to use mask for train/test 
text_sparse_matrix_bigram = vectorizer_bigram.fit_transform(bike_subdf.Noun_adj_parsed_theft_description)    


In [24]:
id2words= dict((v, k) for k, v in vectorizer.vocabulary_.iteritems())
id2words_bigram= dict((v, k) for k, v in vectorizer_bigram.vocabulary_.iteritems())
corpus_gensim = gensim.matutils.Sparse2Corpus(text_sparse_matrix, documents_columns=False)
corpus_gensim_bigram = gensim.matutils.Sparse2Corpus(text_sparse_matrix_bigram, documents_columns=False)

In [25]:
print "Corpus shape of unigram: ", corpus_gensim.sparse.get_shape()
print "Corpus shape of bigram: ", corpus_gensim_bigram.sparse.get_shape()
print "Number of words in vocabulary:", len(vectorizer.vocabulary_)


Corpus shape of unigram:  (1832, 29838)
Corpus shape of bigram:  (4720, 29838)
Number of words in vocabulary: 1832


In [26]:
#Use sparse matrix to initiate K-means clustering


from sklearn.cluster import KMeans

num_clusters = np.arange(2,11)
kmeans_model={}

for i in num_clusters:
    kmeans_model[i] = KMeans(init='k-means++', n_clusters=i, n_init=10)
    %time kmeans_model[i].fit(text_sparse_matrix)



Wall time: 6.67 s
Wall time: 5.37 s
Wall time: 16.7 s
Wall time: 17.3 s
Wall time: 14 s
Wall time: 12 s
Wall time: 16.7 s
Wall time: 14.6 s
Wall time: 17.7 s


In [40]:
#Credit: http://scikit-learn.org/stable/auto_examples/text/document_clustering.html#example-text-document-clustering-py
num_cluster = np.arange(2,11)

for i in num_cluster:
    print "###### Printing Top terms for ", i," number of clusters: ###### "
    order_centroids = kmeans_model[i].cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    for i in range(0,i):
        print("Cluster %d:" % i)
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind])
        print()

###### Printing Top terms for  2  number of clusters: ###### 
Cluster 0:
 bike
 rack
 lock
 garage
 cable
 apartment
 building
 thief
 frame
 car
()
Cluster 1:
 garage
 rack
 home
 apartment
 house
 parking
 porch
 lock
 yard
 building
()
###### Printing Top terms for  3  number of clusters: ###### 
Cluster 0:
 garage
 rack
 home
 apartment
 house
 parking
 porch
 lock
 yard
 building
()
Cluster 1:
 bike
 rack
 lock
 garage
 cable
 apartment
 building
 parking
 car
 thief
()
Cluster 2:
 bike
 lock
 rack
 cable
 garage
 frame
 wheel
 apartment
 thief
 building
()
###### Printing Top terms for  4  number of clusters: ###### 
Cluster 0:
 garage
 bike
 parking
 home
 door
 apartment
 lock
 thief
 building
 house
()
Cluster 1:
 bike
 rack
 lock
 cable
 apartment
 building
 car
 garage
 thief
 parking
()
Cluster 2:
 bike
 lock
 rack
 cable
 frame
 garage
 wheel
 apartment
 thief
 building
()
Cluster 3:
 rack
 home
 apartment
 porch
 house
 yard
 backyard
 lock
 building
 car
()
###### Printi

In [36]:
#TEMP
model_labels = kmeans_model[2].labels_
print len(model_labels)

29838


In [38]:
for i in num_clusters:
    print "Printing cluster centers for Kmeans with ", i, " of clusters"
    print kmeans_model[i].cluster_centers_

Printing cluster centers for Kmeans with  2  of clusters
[[  1.51400454e-04   1.51400454e-04   1.51400454e-04 ...,   7.57002271e-04
    4.54201363e-04   6.05601817e-04]
 [  4.30422244e-05   1.29126673e-04   8.60844488e-05 ...,   1.29126673e-04
    4.30422244e-05   0.00000000e+00]]
Printing cluster centers for Kmeans with  3  of clusters
[[  4.30737422e-05   1.29221227e-04   8.61474845e-05 ...,   1.29221227e-04
    4.30737422e-05   0.00000000e+00]
 [  1.67672703e-04   1.67672703e-04   1.67672703e-04 ...,   8.38363514e-04
    3.35345406e-04   5.03018109e-04]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    1.51975684e-03   1.51975684e-03]]
Printing cluster centers for Kmeans with  4  of clusters
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   4.95417389e-04
    0.00000000e+00   0.00000000e+00]
 [  1.83857327e-04   1.83857327e-04   1.83857327e-04 ...,   7.35429307e-04
    3.67714653e-04   5.51571980e-04]
 [  0.00000000e+00   0.00000000e+00   0.000

In [ ]:
len(kmeans_model[2].cluster_centers_[0])

In [43]:
# Using Cosine similarity instead of euclidean distance
from sklearn.metrics.pairwise import cosine_similarity

#Convert sparse matrix to full array
text_full_matrix = text_sparse_matrix.toarray()

#Cosine is a measure of similarity. So we subtract from 1 to get the distance since max value of cos is 1
%time distance = 1 - cosine_similarity(text_full_matrix)

MemoryError: 

In [ ]:
#Compute MDS for Display of points
templabel = np.arange(0,29838)
from sklearn.manifold import MDS
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)  #*******************  Try changing N_components
#Fit distance (calculated earlier ) to MDS and obtain embedded coordinates
pos = mds.fit_transform(dist)

#Plot graph
xs, ys = pos[:, 0], pos[:, 1]
for x, y, name in zip(xs, ys, names):
    #color = 'orange' if "Austen" in name else 'skyblue'
    plt.scatter(x, y)
    plt.text(x, y, templabel)

plt.show()


In [ ]:
%time linkage_matrix = ward(distance) #define the linkage_matrix using ward clustering pre-computed distances


In [ ]:
fig, ax = plt.subplots(figsize=(15, 20)) # set size
ax = dendrogram(linkage_matrix, orientation="right");

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout